In [1]:
#standard import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import pyreadr

# sklearn
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, HuberRegressor, QuantileRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.utils import resample
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.datasets import make_regression
from sklearn import datasets

# miscilaneous models
from xgboost import XGBRegressor
from quantile_forest import RandomForestQuantileRegressor
from joblib import Parallel, delayed
#from exp_utils import *
import time
from scipy.stats import multivariate_normal
import pickle
from sklearn.neighbors import KNeighborsRegressor
from optbinning import ContinuousOptimalBinning
import openml
from ucimlrepo import fetch_ucirepo 
import os

from imodels.importance import RandomForestPlusRegressor
#import blosc

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

/home/mattyshen/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


(CVXPY) Jun 17 11:02:17 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.10.4067). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jun 17 11:02:17 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.10.4067). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


# Data

In [2]:
def process_data(X, y, cat_feat, num_feat):
    # one-hot encode categorical features
    X_processed = pd.get_dummies(X, columns=cat_feat, drop_first=True, dtype=int)
    
    # save categorical features after one-hot encoding
    cat_feat_dummy = X_processed.drop(columns=num_feat).columns.to_numpy()
    
    # get feature importance
    # random forest
    imp_model_rf = RandomForestRegressor(min_samples_leaf = 5, max_features = 0.33, n_estimators = 100, random_state=777)
    feat_imp_rf = imp_model_rf.fit(X_processed, y).feature_importances_

    #rf+
    print("rf+")
    rf_model = RandomForestRegressor(min_samples_leaf = 5, max_features = 0.33, n_estimators = 100, random_state=777)
    imp_model_rf_plus = RandomForestPlusRegressor(rf_model=rf_model)
    imp_model_rf_plus.fit(X_processed,y)
    feat_imp_rf_plus = imp_model_rf_plus.get_mdi_plus_scores(X_processed,y)

    imp_df = pd.DataFrame({"feature": X_processed.columns,
                           "importance_rf": feat_imp_rf,
                           "importance_rf_plus": feat_imp_rf_plus.importance}).sort_values("importance_rf", ascending=False)    
    # bin data
    X_binned = X_processed[num_feat].apply(lambda c: pd.qcut(c, q=4, duplicates="drop"), axis=0)
    X_binned = pd.concat([X_binned, X_processed[cat_feat_dummy]], axis=1)
    
    subgroup_dict = {"num_feat": num_feat,
                    "cat_feat": cat_feat_dummy,
                    "importance": imp_df,
                    "binned_df": X_binned}
    
    return X_processed, y, subgroup_dict

data = "data_abalone"
# fetch dataset 
abalone = fetch_ucirepo(id=1) 

# data (as pandas dataframes) 
X_orig = abalone.data.features
y = abalone.data.targets.Rings.# Abalone
features: 

target:

source:


In [13]:
data = "data_abalone"
# fetch dataset 
abalone = fetch_ucirepo(id=1) 

# data (as pandas dataframes) 
X_orig = abalone.data.features
y = abalone.data.targets.Rings

In [14]:
X_orig

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550
...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960


In [8]:
pd.DataFrame(X_orig.values, columns=X_orig.columns, index=X_orig.index)

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
0,M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15
1,M,0.35,0.265,0.09,0.2255,0.0995,0.0485,0.07
2,F,0.53,0.42,0.135,0.677,0.2565,0.1415,0.21
3,M,0.44,0.365,0.125,0.516,0.2155,0.114,0.155
4,I,0.33,0.255,0.08,0.205,0.0895,0.0395,0.055
...,...,...,...,...,...,...,...,...
4172,F,0.565,0.45,0.165,0.887,0.37,0.239,0.249
4173,M,0.59,0.44,0.135,0.966,0.439,0.2145,0.2605
4174,M,0.6,0.475,0.205,1.176,0.5255,0.2875,0.308
4175,F,0.625,0.485,0.15,1.0945,0.531,0.261,0.296


In [10]:
np.unique(y)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29])

In [6]:
X_orig = pd.DataFrame(X_orig.values, columns=X_orig.columns, index=X_orig.index)
X_orig

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
0,M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15
1,M,0.35,0.265,0.09,0.2255,0.0995,0.0485,0.07
2,F,0.53,0.42,0.135,0.677,0.2565,0.1415,0.21
3,M,0.44,0.365,0.125,0.516,0.2155,0.114,0.155
4,I,0.33,0.255,0.08,0.205,0.0895,0.0395,0.055
...,...,...,...,...,...,...,...,...
4172,F,0.565,0.45,0.165,0.887,0.37,0.239,0.249
4173,M,0.59,0.44,0.135,0.966,0.439,0.2145,0.2605
4174,M,0.6,0.475,0.205,1.176,0.5255,0.2875,0.308
4175,F,0.625,0.485,0.15,1.0945,0.531,0.261,0.296


In [7]:
data = "data_abalone"
# fetch dataset 
abalone = fetch_ucirepo(id=1) 

# data (as pandas dataframes) 
X_orig = abalone.data.features
y = abalone.data.targets.Rings.to_numpy() 

# specify categorical and numerical features
cat_feat_abalone = ["Sex"]
num_feat_abalone = X_orig.drop(columns = cat_feat_abalone).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_abalone, num_feat_abalone)

bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

rf+


KeyboardInterrupt: 

In [ ]:
# data = "data_abalone"
# # fetch dataset 
# abalone = fetch_ucirepo(id=1) 

# # data (as pandas dataframes) 
# X_orig = abalone.data.features

# # specify categorical and numerical features
# cat_feat_abalone = ["Sex"]
# num_feat_abalone = X_orig.drop(columns = cat_feat_abalone).columns.to_numpy()

# X_processed = pd.get_dummies(X_orig, columns=cat_feat_abalone, drop_first=True, dtype=int)
# X = X_processed

# # bin data
# abalone_binned = X_orig.drop(columns = cat_feat_abalone).apply(lambda c: pd.cut(c, bins=5), axis=0)
# abalone_binned[cat_feat_abalone] = X_orig[cat_feat_abalone]

# abalone_binned_quantile = X_orig.drop(columns = cat_feat_abalone).apply(lambda c: pd.qcut(c, q=4, duplicates="drop"), axis=0)
# abalone_binned_quantile[cat_feat_abalone] = X_orig[cat_feat_abalone]

# y = abalone.data.targets.Rings.to_numpy() 

# Parkinsons

In [95]:
data = "data_parkinsons"
# fetch dataset 
parkinsons_telemonitoring = fetch_ucirepo(id=189) 

# data (as pandas dataframes) 
X = parkinsons_telemonitoring.data.features.drop(columns = "test_time")
y = parkinsons_telemonitoring.data.targets.total_UPDRS

In [98]:
X.dtypes

age                int64
Jitter(%)        float64
Jitter(Abs)      float64
Jitter:RAP       float64
Jitter:PPQ5      float64
Jitter:DDP       float64
Shimmer          float64
Shimmer(dB)      float64
Shimmer:APQ3     float64
Shimmer:APQ5     float64
Shimmer:APQ11    float64
Shimmer:DDA      float64
NHR              float64
HNR              float64
RPDE             float64
DFA              float64
PPE              float64
sex                int64
dtype: object

In [19]:
y

0       34.398
1       34.894
2       35.389
3       35.810
4       36.375
         ...  
5870    33.485
5871    32.988
5872    32.495
5873    32.007
5874    31.513
Name: total_UPDRS, Length: 5875, dtype: float64

In [ ]:
data = "data_parkinsons"
# fetch dataset 
parkinsons_telemonitoring = fetch_ucirepo(id=189) 

# data (as pandas dataframes) 
X_orig = parkinsons_telemonitoring.data.features.drop(columns = "test_time")
y = parkinsons_telemonitoring.data.targets.total_UPDRS.to_numpy()

In [ ]:
sns.set_context("talk", font_scale=1.5)
plt.hist(X_orig.DFA, bins=50, color="grey")
plt.axvline(0.6827888, color="black", linestyle="dashed")
plt.xlabel("Speech Oscillation")

In [ ]:
data = "data_parkinsons"
# fetch dataset 
parkinsons_telemonitoring = fetch_ucirepo(id=189) 

# data (as pandas dataframes) 
X_orig = parkinsons_telemonitoring.data.features.drop(columns = "test_time")
y = parkinsons_telemonitoring.data.targets.total_UPDRS.to_numpy()

# save categorical and numerical features
cat_feat_parkinsons = ["sex"]
num_feat_parkinsons = X_orig.drop(columns = cat_feat_parkinsons).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_parkinsons, num_feat_parkinsons)
bin_df = subgroups["binned_df"]

new_bin_df = bin_df.copy(deep=True)
new_bin_df["DFA"] = (X.DFA <= 0.68).astype(int)
subgroups["new_binned_df"] = new_bin_df

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, x_new_bin_train_val, x_new_bin_test, y_train_val, y_test = train_test_split(X, bin_df, new_bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, x_new_bin_train, x_new_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, x_new_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True),
                  "x_new_bin_train": x_new_bin_train.reset_index(drop=True),
                  "x_new_bin_val": x_new_bin_val.reset_index(drop=True),
                  "x_new_bin_test": x_new_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig_new.dat", "wb") as f:
    f.write(pickle_compressed)

In [27]:
data = "data_airfoil"
# fetch dataset 
airfoil_self_noise = fetch_ucirepo(id=291) 
  
# data (as pandas dataframes) 
X_orig = airfoil_self_noise.data.features 
y = airfoil_self_noise.data.targets["scaled-sound-pressure"]

In [28]:
y

0       126.201
1       125.201
2       125.951
3       127.591
4       127.461
         ...   
1498    110.264
1499    109.254
1500    106.604
1501    106.224
1502    104.204
Name: scaled-sound-pressure, Length: 1503, dtype: float64

# Airfoil

In [ ]:
data = "data_airfoil"
# fetch dataset 
airfoil_self_noise = fetch_ucirepo(id=291) 
  
# data (as pandas dataframes) 
X_orig = airfoil_self_noise.data.features 
y = airfoil_self_noise.data.targets["scaled-sound-pressure"].to_numpy()

# save categorical and numerical features
cat_feat_airfoil = []
num_feat_airfoil = X_orig.drop(columns = cat_feat_airfoil).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_airfoil, num_feat_airfoil)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

# Computer

In [36]:
computer = openml.datasets.get_dataset(197)

# data
X_orig, y, _, _ = computer.get_data(target=computer.default_target_attribute, dataset_format="dataframe")

In [39]:
y

0       90
1       88
2       85
3       81
4       79
        ..
8187    69
8188    88
8189    92
8190    96
8191    80
Name: usr, Length: 8192, dtype: uint8

In [30]:
X_orig

,lread,lwrite,scall,sread,swrite,fork,exec,rchar,wchar,pgout,...,pgfree,pgscan,atch,pgin,ppgin,pflt,vflt,runqsz,freemem,freeswap
0,6.0,2.0,1036.0,103.0,114.0,1.00,1.00,172076.0,355965.0,0.00,...,0.00,0.00,0.00,2.00,4.00,73.60,89.00,2.0,6527.0,1851864.0
1,1.0,0.0,2165.0,205.0,101.0,0.40,1.20,43107.0,44139.0,4.80,...,75.80,181.40,0.20,85.40,88.20,19.40,161.80,3.0,130.0,1131931.0
2,62.0,77.0,3806.0,258.0,166.0,1.40,1.40,492142.0,268706.0,4.80,...,44.00,79.20,2.20,7.60,12.20,68.00,218.80,5.2,256.0,1314590.0
3,5.0,0.0,4721.0,256.0,177.0,0.99,2.58,524787.0,174964.0,14.51,...,88.47,189.86,1.99,4.17,24.85,95.63,248.91,1.0,233.0,972606.0
4,42.0,55.0,3949.0,249.0,244.0,2.60,4.60,197289.0,529200.0,4.20,...,6.60,0.00,1.40,1.80,2.20,219.60,297.20,3.4,331.0,1013805.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,74.0,49.0,2688.0,176.0,103.0,11.00,32.20,57714.0,38484.0,0.80,...,1.00,0.00,0.00,0.80,0.80,343.20,649.40,7.0,314.0,1096333.0
8188,29.0,40.0,1906.0,118.0,90.0,0.80,2.00,8175.0,27313.0,0.00,...,0.00,0.00,0.00,0.80,0.80,56.20,78.60,3.6,166.0,1107088.0
8189,3.0,0.0,926.0,90.0,67.0,0.60,1.00,5411.0,19322.0,0.00,...,0.00,0.00,0.40,0.40,0.40,53.40,154.00,1.0,1177.0,1020400.0
8190,4.0,0.0,418.0,30.0,29.0,0.80,1.00,3959.0,10679.0,0.00,...,0.00,0.00,0.00,0.20,0.20,61.00,73.20,2.4,6355.0,1702592.0


In [ ]:
data = "data_computer"

# fetch dataset
computer = openml.datasets.get_dataset(197)

# data
X_orig, y, cat_ind, col_names = computer.get_data(target=computer.default_target_attribute, dataset_format="dataframe")
y = y.to_numpy()

# save categorical and numerical features
cat_feat_computer = []
num_feat_computer = X_orig.drop(columns = cat_feat_computer).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_computer, num_feat_computer)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

# Concrete

In [44]:
data = "data_concrete"

# fetch dataset 
concrete_compressive_strength = fetch_ucirepo(id=165) 
  
# data (as pandas dataframes) 
X_orig = concrete_compressive_strength.data.features 
y = concrete_compressive_strength.data.targets["Concrete compressive strength"]

In [45]:
y

0       79.99
1       61.89
2       40.27
3       41.05
4       44.30
        ...  
1025    44.28
1026    31.18
1027    23.70
1028    32.77
1029    32.40
Name: Concrete compressive strength, Length: 1030, dtype: float64

In [ ]:
data = "data_concrete"

# fetch dataset 
concrete_compressive_strength = fetch_ucirepo(id=165) 
  
# data (as pandas dataframes) 
X_orig = concrete_compressive_strength.data.features 
y = concrete_compressive_strength.data.targets["Concrete compressive strength"].to_numpy()

# save categorical and numerical features
cat_feat_concrete = []
num_feat_concrete = X_orig.drop(columns = cat_feat_concrete).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_concrete, num_feat_concrete)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

# Powerplant

In [46]:
data = "data_powerplant"
combined_cycle_power_plant = fetch_ucirepo(id=294) 
X = combined_cycle_power_plant.data.features 
y = combined_cycle_power_plant.data.targets.PE

In [48]:
y

0       463.26
1       444.37
2       488.56
3       446.48
4       473.90
         ...  
9563    460.03
9564    469.62
9565    429.57
9566    435.74
9567    453.28
Name: PE, Length: 9568, dtype: float64

In [ ]:
data = "data_powerplant"

# fetch dataset 
combined_cycle_power_plant = fetch_ucirepo(id=294) 
  
# data (as pandas dataframes) 
X_orig = combined_cycle_power_plant.data.features 
y = combined_cycle_power_plant.data.targets.PE.to_numpy()

# save categorical and numerical features
cat_feat_powerplant = []
num_feat_powerplant = X_orig.drop(columns = cat_feat_powerplant).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_powerplant, num_feat_powerplant)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

# Miami Housing

In [53]:
# fetch dataset
miami_housing = openml.datasets.get_dataset(43093)

# data
X, y, _, _ = miami_housing.get_data(target=miami_housing.default_target_attribute, dataset_format="dataframe")

In [54]:
y

0        440000.0
1        349000.0
2        800000.0
3        988000.0
4        755000.0
           ...   
13927    275000.0
13928    340000.0
13929    287500.0
13930    315000.0
13931    250000.0
Name: SALE_PRC, Length: 13932, dtype: float64

In [ ]:
np.mean(X_orig["WATER_DIST"]<12700)

In [ ]:
plt.hist(X_orig["OCEAN_DIST"], bins=50)

In [ ]:
plt.hist(X_orig["WATER_DIST"], bins=100)
plt.axvline(12700, color="red")

In [ ]:
sns.set_context("talk", font_scale=1.5)
plt.hist(X_orig.WATER_DIST, bins=50, color="grey")
plt.axvline(12700, color="black", linestyle="dashed")
plt.xlabel("Distance to Water")

In [ ]:
data = "data_miami_housing"

# fetch dataset
miami_housing = openml.datasets.get_dataset(43093)

# data
X_orig, y, cat_ind, col_names = miami_housing.get_data(target=miami_housing.default_target_attribute, dataset_format="dataframe")
X_orig, y = resample(X_orig, y, replace=False, n_samples=5000, random_state=777)
X_orig = X_orig.drop(columns="PARCELNO").reset_index(drop=True)
y = y.to_numpy()

# save categorical and numerical features
cat_feat_miami_housing = ["avno60plus", "month_sold", "structure_quality"]
num_feat_miami_housing = X_orig.drop(columns = cat_feat_miami_housing).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_miami_housing, num_feat_miami_housing)
bin_df = subgroups["binned_df"]

new_bin_df = bin_df.copy(deep=True)
new_bin_df["WATER_DIST"] = (X.WATER_DIST <= 12700).astype(int)
subgroups["new_binned_df"] = new_bin_df


data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, x_new_bin_train_val, x_new_bin_test, y_train_val, y_test = train_test_split(X, bin_df, new_bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, x_new_bin_train, x_new_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, x_new_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True),
                  "x_new_bin_train": x_new_bin_train.reset_index(drop=True),
                  "x_new_bin_val": x_new_bin_val.reset_index(drop=True),
                  "x_new_bin_test": x_new_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig_new.dat", "wb") as f:
    f.write(pickle_compressed)

In [55]:
data = "data_traffic"
metro_interstate_traffic_volume = fetch_ucirepo(id=492) 
X = metro_interstate_traffic_volume.data.features 
y = metro_interstate_traffic_volume.data.targets.traffic_volume

In [62]:
len(y)

48204

In [57]:
X['holiday'].value_counts()

holiday
Labor Day                    7
Thanksgiving Day             6
Christmas Day                6
New Years Day                6
Martin Luther King Jr Day    6
Columbus Day                 5
Veterans Day                 5
Washingtons Birthday         5
Memorial Day                 5
Independence Day             5
State Fair                   5
Name: count, dtype: int64

# Traffic

In [ ]:
data = "data_traffic"

# fetch dataset 
metro_interstate_traffic_volume = fetch_ucirepo(id=492) 
  
# data (as pandas dataframes) 
X_orig = metro_interstate_traffic_volume.data.features 
y = metro_interstate_traffic_volume.data.targets.traffic_volume.to_numpy() 

# subset data
X_orig["timestamp"] = pd.to_datetime(X_orig.date_time)

# select only 2014 data
year_selector = (X_orig.timestamp.dt.year >= 2013) & (X_orig.timestamp.dt.year <= 2017) 
X_orig = X_orig[year_selector]
y = y[year_selector]

# selec only 8 am data
hour_selector = (X_orig.timestamp.dt.hour >= 7) & (X_orig.timestamp.dt.hour <= 9)
X_orig = X_orig[hour_selector].reset_index(drop=True)
y = y[hour_selector]

# clean data
X_orig = X_orig.drop(columns = ["weather_description", "date_time", "holiday"])
X_orig["year"] = X_orig.timestamp.dt.year
X_orig["month"] = X_orig.timestamp.dt.month
X_orig["hour"] = X_orig.timestamp.dt.hour
X_orig = X_orig.drop(columns = ["timestamp"])

# save categorical and numerical features
cat_feat_traffic = ["weather_main", "year", "month", "hour"]
num_feat_traffic = X_orig.drop(columns = cat_feat_traffic).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_traffic, num_feat_traffic)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

# Insurance

In [81]:
data = "data_insurance"

# fetch data
insurance = pd.read_csv("https://raw.githubusercontent.com/pycaret/datasets/main/data/common/insurance.csv")

# data
X_orig = insurance.drop(columns="charges")
y = insurance.charges

In [82]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

In [ ]:
data = "data_insurance"

# fetch data
insurance = pd.read_csv("https://raw.githubusercontent.com/pycaret/datasets/main/data/common/insurance.csv")

# data
X_orig = insurance.drop(columns="charges")
y = insurance.charges.to_numpy()

# save categorical and numerical features
cat_feat_insurance = ["sex", "smoker", "region"]
num_feat_insurance = X_orig.drop(columns = cat_feat_insurance).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_insurance, num_feat_insurance)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

# CA Housing

In [ ]:
data = "data_ca_housing"

# fetch data
housing = pd.read_csv("../../data/" + "cal_housing.data", delimiter=",", names = ["lon", "lat", "med_age", "total_rooms", "total_beds", "population", "households", "med_income", "med_price"])

# data
X_orig = housing.drop(columns="med_price")
y = housing.med_price.to_numpy()

X_orig, y = resample(X_orig, y, replace=False, n_samples=5000, random_state=777)
X_orig = X_orig.reset_index(drop=True)

# save categorical and numerical features
cat_feat_ca_housing = []
num_feat_ca_housing = X_orig.drop(columns = cat_feat_ca_housing).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_ca_housing, num_feat_ca_housing)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

# fetch dataset
qsar = openml.datasets.get_dataset(4048)

# data
X_orig, y, cat_ind, col_names = qsar.get_data(target=qsar.default_target_attribute, dataset_format="dataframe")
y = y.to_numpy()# QSAR

In [73]:
# fetch dataset
qsar = openml.datasets.get_dataset(4048)

# data
X, y, cat_ind, col_names = qsar.get_data(target=qsar.default_target_attribute, dataset_format="dataframe")

In [78]:
X

,FCFP6_1024_0,FCFP6_1024_1,FCFP6_1024_2,FCFP6_1024_3,FCFP6_1024_4,FCFP6_1024_5,FCFP6_1024_6,FCFP6_1024_7,FCFP6_1024_8,FCFP6_1024_9,...,FCFP6_1024_1014,FCFP6_1024_1015,FCFP6_1024_1016,FCFP6_1024_1017,FCFP6_1024_1018,FCFP6_1024_1019,FCFP6_1024_1020,FCFP6_1024_1021,FCFP6_1024_1022,FCFP6_1024_1023
0,1.0,1.0,0,1.0,0,0,0,0,0,1.0,...,0,1.0,0,0,1.0,0,0,0,0,0
1,1.0,1.0,0,1.0,0,0,0,1.0,0,1.0,...,0,0,0,0,1.0,1.0,0,0,0,0
2,1.0,1.0,0,1.0,0,0,0,0,0,1.0,...,0,0,0,0,1.0,1.0,0,0,0,0
3,1.0,1.0,0,1.0,0,0,0,0,0,1.0,...,0,0,0,0,1.0,1.0,0,0,0,0
4,1.0,1.0,0,1.0,0,0,0,0,0,1.0,...,0,0,0,0,1.0,1.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,1.0,1.0,0,1.0,0,0,0,1.0,1.0,0,...,0,0,0,0,0,0,0,1.0,0,0
1170,1.0,1.0,0,1.0,0,0,0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1171,1.0,1.0,0,1.0,0,0,0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1172,1.0,1.0,0,1.0,0,0,0,1.0,1.0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# fetch dataset
qsar = openml.datasets.get_dataset(4048)

# data
X_orig, y, cat_ind, col_names = qsar.get_data(target=qsar.default_target_attribute, dataset_format="dataframe")
y = y.to_numpy()

In [ ]:
plt.hist(y)

In [ ]:
data = "data_qsar"

# fetch data
qsar = openml.tasks.get_task(360932)

# data
X_orig, y = qsar.get_X_and_y(dataset_format="dataframe")
y = y.to_numpy()

# select features with variance
k = 500
X_array = X_orig.to_numpy()
top_k_columns = X_orig.columns[np.argsort(np.var(X_array, axis=0))[-k:]]
X_orig = X_orig[top_k_columns]

X_orig, y= resample(X_orig, y, replace=False, n_samples=5000, random_state=777)

# save categorical and numerical features
cat_feat_qsar = X_orig.columns.tolist()
num_feat_qsar = X_orig.drop(columns = cat_feat_qsar).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_qsar, num_feat_qsar)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

# Allstate

In [83]:
data = "data_allstate"
allstate = openml.datasets.get_dataset(42571)
X, y, _, _ = allstate.get_data(target=allstate.default_target_attribute, dataset_format="dataframe")

In [85]:
y

0         2213.18
1         1283.60
2         3005.09
3          939.85
4         2763.85
           ...   
188313    1198.62
188314    1108.34
188315    5762.64
188316    1562.87
188317    4751.72
Name: loss, Length: 188318, dtype: float64

In [ ]:
cat_ind

In [ ]:
data = "data_allstate"

# fetch dataset
allstate = openml.datasets.get_dataset(42571)

# data
X_orig, y, cat_ind, col_names = allstate.get_data(target=allstate.default_target_attribute, dataset_format="dataframe")
y = y.to_numpy()

X_orig, y = resample(X_orig, y, replace=False, n_samples=5000, random_state=777)

# save categorical and numerical features
cat_feat_allstate = np.array(col_names)[cat_ind].tolist()
num_feat_allstate = X_orig.drop(columns = cat_feat_allstate).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_allstate, num_feat_allstate)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

# Mercedes

In [88]:
data = "data_mercedes"
mercedes = openml.datasets.get_dataset(42570)
X, y, cat_ind, col_names = mercedes.get_data(target=mercedes.default_target_attribute, dataset_format="dataframe")

In [89]:
type(X), type(y),

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [92]:
len(X.columns)

376

In [ ]:
data = "data_mercedes"

# fetch dataset
mercedes = openml.datasets.get_dataset(42570)

# data
X_orig, y, cat_ind, col_names = mercedes.get_data(target=mercedes.default_target_attribute, dataset_format="dataframe")
y = y.to_numpy()

# save categorical and numerical features
cat_feat_mercedes = X_orig.columns.tolist()
num_feat_mercedes = X_orig.drop(columns = cat_feat_mercedes).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_mercedes, num_feat_mercedes)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

# Transaction

In [93]:
data = "data_transaction"
transaction = openml.datasets.get_dataset(42572)
X, y, _, _ = transaction.get_data(target=transaction.default_target_attribute, dataset_format="dataframe")

In [94]:
y

0       38000000.0
1         600000.0
2       10000000.0
3        2000000.0
4       14400000.0
           ...    
4454     1065000.0
4455       48000.0
4456     2800000.0
4457    10000000.0
4458    20000000.0
Name: target, Length: 4459, dtype: float64

In [ ]:
data = "data_transaction"

# fetch dataset
transaction = openml.datasets.get_dataset(42572)

# data
X_orig, y, cat_ind, col_names = transaction.get_data(target=transaction.default_target_attribute, dataset_format="dataframe")

# select features with variance
k=500
X_variance = X_orig.var()
top_k_columns = X_variance.sort_values(ascending=False).head(k).index
X_orig = X_orig[top_k_columns]

y = y.to_numpy()

# save categorical and numerical features
cat_feat_transaction = []
num_feat_transaction = X_orig.drop(columns = cat_feat_transaction).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_transaction, num_feat_transaction)
bin_df = subgroups["binned_df"]

In [ ]:
data = "data_transaction"

# fetch dataset
transaction = openml.datasets.get_dataset(42572)

# data
X_orig, y, cat_ind, col_names = transaction.get_data(target=transaction.default_target_attribute, dataset_format="dataframe")

# select features with variance
k=500
X_variance = X_orig.var()
top_k_columns = X_variance.sort_values(ascending=False).head(k).index
X_orig = X_orig[top_k_columns]

y = y.to_numpy()

# save categorical and numerical features
cat_feat_transaction = []
num_feat_transaction = X_orig.drop(columns = cat_feat_transaction).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_transaction, num_feat_transaction)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

# fMRI

In [ ]:
data = "data_fMRI"

# data
X_orig = pd.read_csv("../../data/fmri/X.csv")
y = pd.read_csv("../../data/fmri/Y.csv")

In [ ]:
y

In [ ]:
37400 / 1870

In [ ]:
k=500
X_variance = X_orig.var()
top_k_columns = X_variance.sort_values(ascending=False).head(k).index
X_orig = X_orig[top_k_columns]
X_orig

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_orig, y, test_size=0.2)

In [ ]:
model = LassoCV()
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)
r2_score(y_test, y_pred)

In [ ]:
data = "data_fMRI"

# data
X_orig = pd.read_csv("../../data/fmri/X.csv")
y = pd.read_csv("../../data/fmri/Y.csv").iloc[:X_orig.shape[0], 0]

# select features with variance
k=500
X_variance = X_orig.var()
top_k_columns = X_variance.sort_values(ascending=False).head(k).index
X_orig = X_orig[top_k_columns]

y = y.to_numpy()

# save categorical and numerical features
cat_feat_fmri = []
num_feat_fmri = X_orig.drop(columns = cat_feat_fmri).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_fmri, num_feat_fmri)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig.dat", "wb") as f:
    f.write(pickle_compressed)

# CCLE

In [ ]:
X_orig = pd.read_csv("../../data/ccle/X.csv")
y = pd.read_csv("../../data/ccle/Y.csv").loc[:,"Erlotinib"]

In [ ]:
y

In [ ]:
X_orig = pd.read_csv("../../data/ccle/X.csv")
y = pd.read_csv("../../data/ccle/Y.csv").loc[:,"Erlotinib"]

k=500
X_variance = X_orig.var()
top_k_columns = X_variance.sort_values(ascending=False).head(k).index
X_orig = X_orig[top_k_columns]

model = RidgeCV(alphas=[1,10,100, 1000, 10000])
model.fit(X_orig, y)

In [ ]:
y_pred = model.predict(X_orig)
r2_score(y, y_pred)

In [ ]:
model.intercept_

In [ ]:
data = "data_ccle"

# data
X_orig = pd.read_csv("../../data/ccle/X.csv")
y = pd.read_csv("../../data/ccle/Y.csv").loc[:,"AEW541"]

# select features with variance
k=300
X_variance = X_orig.var()
top_k_columns = X_variance.sort_values(ascending=False).head(k).index
X_orig = X_orig[top_k_columns]

y = y.to_numpy()

# save categorical and numerical features
cat_feat_ccle = []
num_feat_ccle = X_orig.drop(columns = cat_feat_ccle).columns.to_numpy()

X, y, subgroups = process_data(X_orig, y, cat_feat_ccle, num_feat_ccle)
bin_df = subgroups["binned_df"]

data_dict = {}

for i in range(5):
    np.random.seed(777+i)
    x_train_val, x_test, x_bin_train_val, x_bin_test, y_train_val, y_test = train_test_split(X, bin_df, y, test_size=0.2)
    x_train, x_val, x_bin_train, x_bin_val, y_train, y_val = train_test_split(x_train_val, x_bin_train_val, y_train_val, test_size=0.25)
    
    split_dict = {"x_train": x_train.to_numpy(), "y_train": y_train,
                  "x_val": x_val.to_numpy(), "y_val": y_val,
                  "x_test": x_test.to_numpy(), "y_test": y_test,
                  "x_bin_train": x_bin_train.reset_index(drop=True),
                  "x_bin_val": x_bin_val.reset_index(drop=True),
                  "x_bin_test": x_bin_test.reset_index(drop=True)}
    
    data_dict[str(i)] = {"orig": split_dict}

data_dict["subgroup_info"] = subgroups
data_dict["X"] = X
data_dict["y"] = y

if not os.path.exists(f"../data/{data}"):
    os.mkdir(f"../data/{data}")
    
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_orig_300.dat", "wb") as f:
    f.write(pickle_compressed)

# Enhancer

In [ ]:
data = "data_enhancer"
with open(f"../data/{data}/data_groups_0.25.pkl", "rb") as f:
    data_dict = pickle.load(f)

In [ ]:
data_dict_pickled = pickle.dumps(data_dict)
pickle_compressed = blosc.compress(data_dict_pickled)

with open(f"../data/{data}/data_groups_0.25.dat", "wb") as f:
    f.write(pickle_compressed)

In [ ]:
liver = openml.datasets.get_dataset(200)

# data
X_orig, y, cat_ind, col_names = liver.get_data(target=liver.default_target_attribute, dataset_format="dataframe")

In [ ]:
X_orig

In [ ]:
y